#### Feature Engineering & Encoding.

Esto lo ponemos en un notebook aparte porque a partir de acá se crearán Pipelines para los datos, además, se crearán los artifacts necesarios a usar en el despliegue.

**Feature Engineering tasks:** 

- Geohashes. **DONE**
- Precio por m2 en Geohashes (KNN para los por inputar)
- Diferencia entre area total y area construida. **DONE**
- Qué tanto difiere el tamaño de casa con el promedio del distrito. 

**Hacer encodings**

**Ver Correlaciones**

**Ver distribuciones por categorías**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import geohash2 as gh #pip install geohash2
import gc

lima_data = pd.read_csv('../data/2020_Notebook01_clean_data.csv')
lima_data.describe()

#Las coordenadas son negativas, se scrapearon sin el -1
lima_data['latitud'] = lima_data['latitud']*-1
lima_data['longitud'] = lima_data['longitud']*-1

#### Feature Engineering

In [2]:
lima_data['areas_diff'] = lima_data['Area_total'] - lima_data['Area_constr']

#### GeoHashes

In [3]:
for num in range(5, 10):
    lima_data['geohash_grado{}'.format(num)] = lima_data.apply(lambda x: gh.encode(x['latitud'], x['longitud'], precision = num),axis=1)

In [4]:
top_geohashes = lima_data['geohash_grado7'].value_counts().iloc[0:7].keys().tolist()

#En qué Distrito están los barrios con mayor oferta?
lima_data[lima_data['geohash_grado7'].isin(top_geohashes)]['Barrio'].value_counts()

La Molina            261
Santiago de Surco     32
San Isidro            28
Miraflores             8
NoEspecifica           6
Lima Cercado           1
Ate Vitarte            1
Barranco               1
San Borja              1
La Victoria            1
Name: Barrio, dtype: int64

#### OJO! Varios anuncios no ponen las coordenadas exactas, sino unas aledañas. Por eso se ve como si este barrio tuviera varias casas en oferta. Son las mismas 78 coordenadas. Pero diferentes casas. En la Molina toman de referencia el Molicentro. Igual pasa en Surco y en San isidro pero con algo de menos frecuencia. 

In [5]:
lima_data[lima_data['geohash_grado7'] == top_geohashes[0]][['latitud','longitud']].value_counts()

latitud     longitud  
-12.096333  -77.028072    88
dtype: int64

In [6]:
print(lima_data[lima_data['geohash_grado7'] == top_geohashes[1]][['latitud','longitud']].value_counts())
print(lima_data[lima_data['geohash_grado7'] == top_geohashes[2]][['latitud','longitud']].value_counts())

latitud     longitud  
-12.082020  -76.928234    78
-12.081859  -76.928393     2
            -76.928393     2
-12.082020  -76.928234     2
-12.081075  -76.928427     1
-12.081714  -76.927820     1
-12.081852  -76.928062     1
-12.082040  -76.928287     1
dtype: int64
latitud     longitud  
-12.075639  -76.911006    25
            -76.911006    12
dtype: int64


In [7]:
lima_data.filter(regex='^(geohash*)').describe()    

,geohash_grado5,geohash_grado6,geohash_grado7,geohash_grado8,geohash_grado9
count,5526,5526,5526,5526,5526
unique,127,770,2825,3815,3947
top,6mc73,6mc732,6mc5pjv,6mc5pjvv,6mc5pjvvg
freq,775,144,88,88,88


#### Correlaciones

<AxesSubplot:>

In [8]:
import seaborn as sns
corr = lima_data.corr()
sns.heatmap(corr)
#Las cocheras están correlacionadas con el Precio???

##### Selección del Target.

Tenemos 4 formas de target:

- Precio: Precio total del inmueble.

- Precio_m2_total: Precio / metros cuadrados del area total.

- Precio_m2_constr: Precio / metros cuadrados del area construida.

- Precio_cat: Precio por categorias.

In [9]:
X = lima_data.drop(lima_data.filter(regex='(P?p?recio)').columns,axis=1)
Y = lima_data.filter(regex='(P?p?recio)')
print('columnas Y: {}'.format(Y.columns.values))

columnas Y: ['Precio' 'Precio_m2_total' 'Precio_m2_constr' 'Precio_cat']


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y['Precio_m2_total'], test_size=0.1, random_state = 215)

In [11]:
pd.concat([X_train,y_train],axis=1).to_csv('../data/train.csv',index=False)

pd.concat([X_test,y_test],axis=1).to_csv('../data/test.csv',index=False)